In [2]:
import numpy as np
import pandas as pd

In [3]:
# load the ratings CSV files
df1_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/1.csv')
df2_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/2.csv')
df3_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/3.csv')
df4_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/4.csv')
df5_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/5.csv')
df6_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/6.csv')
df7_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/7.csv')
df8_ratings = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/ratings/8.csv')

In [4]:
# concatenate the ratings dataframes into a single dataframe
df = pd.concat([df1_ratings, df2_ratings, df3_ratings, df4_ratings, 
                df5_ratings, df6_ratings, df7_ratings, df8_ratings])

In [5]:
# load the remaining CSV files
df_amazon = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/amazon_prime_titles.csv')
df_disney = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/disney_plus_titles.csv')
df_hulu = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/hulu_titles.csv')
df_netflix = pd.read_csv('C:/Users/Santiago/Desktop/MLOpsReviews/netflix_titles.csv')

In [6]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [7]:
df.isnull().sum()

userId       0
rating       0
timestamp    0
movieId      0
dtype: int64

In [8]:
df.head()

,userId,rating,timestamp,movieId
0,1,1.00,1425941529,as680
1,1,4.50,1425942435,ns2186
2,1,5.00,1425941523,hs2381
3,1,5.00,1425941546,ns3663
4,1,5.00,1425941556,as9500


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11024289 entries, 0 to 1499999
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   rating     float64
 2   timestamp  int64  
 3   movieId    object 
dtypes: float64(1), int64(2), object(1)
memory usage: 420.5+ MB


In [10]:
df.dtypes

userId         int64
rating       float64
timestamp      int64
movieId       object
dtype: object

In [11]:
df.describe()

,userId,rating,timestamp
count,11024289.00,11024289.00,11024289.00
mean,89972.51,3.53,1172541513.59
std,86866.01,1.06,205677385.96
min,1.00,0.50,789652004.00
25%,28558.00,3.00,990159330.00
50%,56843.00,3.50,1153610452.00
75%,116867.00,4.00,1360469934.00
max,270896.00,5.00,1501826675.00


In [12]:
df.duplicated().sum()
df.drop_duplicates(inplace=True)

In [13]:
np.sort(df['rating'].unique())

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [14]:
df = df.drop('timestamp', axis=1)

In [15]:
df.groupby('movieId')['rating'].mean()

movieId
as1      3.47
as10     3.44
as100    3.61
as1000   3.56
as1001   3.59
         ... 
ns995    3.52
ns996    3.63
ns997    3.53
ns998    3.58
ns999    3.49
Name: rating, Length: 22998, dtype: float64

Second group of datasets

In [16]:
# Add platform column to each DataFrame
df_amazon['platform'] = 'amazon'
df_disney['platform'] = 'disney'
df_hulu['platform'] = 'hulu'
df_netflix['platform'] = 'netflix'

In [17]:
# Concatenate the DataFrames into a single DataFrame
df_platforms = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index=True)

In [18]:
df_platforms.isna().sum()

show_id             0
type                0
title               0
director         8260
cast             5321
country         11499
date_added       9554
release_year        0
rating            864
duration          482
listed_in           0
description         4
platform            0
dtype: int64

In [19]:
df_platforms = df_platforms.drop(['date_added', 'release_year', 'description'], axis=1)

In [20]:
df_platforms.insert(loc=0, column='id', value = df_platforms.apply(lambda x: x['platform'][0] + x['show_id'], axis=1))

In [21]:
df_platforms = df_platforms.drop('show_id', axis=1)

In [22]:
df_platforms.iloc[:] = df_platforms.iloc[:].applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [23]:
df_platforms[['duration_int', 'duration_type']] = df_platforms['duration'].str.split(expand=True)
df_platforms['duration_int'] = df_platforms['duration_int'].fillna(0).astype(int)

In [24]:
df_platforms = df_platforms.drop('duration', axis=1)

In [25]:
df_platforms.describe()

,duration_int
count,22998.00
mean,65.71
std,51.76
min,0.00
25%,3.00
50%,83.00
75%,101.00
max,601.00


In [26]:
df = df.rename(columns={'movieId': 'id'})

In [27]:
average = df.groupby('id')['rating'].mean()

In [28]:
df = average.reset_index()[['id', 'rating']]

In [29]:
df = df.rename(columns={'rating': 'score'})

In [30]:
df = pd.merge(df_platforms, df, on='id')

In [33]:
df['duration_type'] = df['duration_type'].replace('seasons', 'season')

In [38]:
for col in df.columns:
    print(f"{col}: {df[col].nunique()}")

id: 22998
type: 2
title: 22042
director: 10094
cast: 16744
country: 886
rating: 105
listed_in: 1687
platform: 4
duration_int: 225
duration_type: 2
score: 11378


In [52]:
df = df.drop(['id', 'platform', 'duration_int', 'duration_type'], axis=1)

In [ ]:
# Group the rows by title and apply a function to select the row with the least NaN values
df = df.groupby('title', as_index=False).apply(lambda x: x.loc[x.isna().sum(axis=1).idxmin()])

# Drop any remaining duplicate rows
df.drop_duplicates(subset='title', inplace=True)

In [59]:
df

,type,title,director,cast,country,rating,listed_in,score
0,movie,"""mixed up""",nishi chawla,"uday krishna, bethany rishell, lucy bond, sanj...",NaN,16+,"drama, romance",3.47
1,tv show,"""the paramedic angel""",NaN,"nate reidnauer, nikki hrichak, nina randazzo, ...",NaN,all,drama,3.59
2,movie,#alive,cho il,"yoo ah-in, park shin-hye",south korea,tv-ma,"horror movies, international movies, thrillers",3.56
3,movie,#annefrank - parallel stories,"sabina fedeli, anna migotto","helen mirren, gengher gatti",italy,tv-14,"documentaries, international movies",3.56
4,tv show,#blackaf,NaN,"kenya barris, rashida jones, iman benson, genn...",united states,tv-ma,tv comedies,3.59
...,...,...,...,...,...,...,...,...
22037,tv show,忍者ハットリくん,NaN,NaN,japan,tv-y7,"anime series, kids' tv",3.63
22038,tv show,海的儿子,NaN,"li nanxing, christopher lee, jesseca liu, appl...",NaN,tv-14,"international tv shows, tv dramas",3.51
22039,tv show,마녀사냥,NaN,"si-kyung sung, se-yoon yoo, dong-yup shin, ji-...",south korea,tv-ma,"international tv shows, korean tv shows, stand...",3.60
22040,movie,반드시 잡는다,hong-seon kim,baek yoon-sik,south korea,tv-ma,"dramas, international movies, thrillers",3.57
